In [ ]:
%%capture
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score
#%pip install git+https://github.com/huggingface/nlp.git@fix-bad-type-in-overflow-check

In [ ]:
QPATH = "Quantlet/4-qode2desc"

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

In [ ]:
import pickle
import json
import re
import sys
from IPython.display import display
import datetime

from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
nltk.download('punkt')
import evaluate

from sklearn.model_selection import train_test_split

import importlib
import analysis_modules
importlib.reload(analysis_modules)

import torch, gc

In [ ]:
#model_name = '../4-qode2desc/no_bootstrap/analysis_report_CodeT5_domain_checkpoint-12290-test-10-512-16-2023-10-07/results/checkpoint-2500'
# no domain model

model_name = '../4-qode2desc/no_bootstrap/analysis_report_CodeT5-no_context-test-10-512-16-2023-10-07/results/checkpoint-2500'
SAMPLE = 'test'
if SAMPLE == 'test':
  load_best_model_at_end=False
else:
  load_best_model_at_end=None

# tokenization
encoder_max_length = 512
decoder_max_length = 150
RS = 42
LR = 5e-5

EPOCHS = 10
TRAIN_BATCH = 16
EVAL_BATCH  = 4

WARMUP_STEPS  = 500
WEIGHT_DECAY  = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 1
SAVE_STRATEGY = 'no'

LABEL_SMOOTHING  = 0.1
PREDICT_GENERATE = True

EVAL_COLUMNS = ['eval_loss',
                'eval_rouge1',
                'eval_rouge2',
                'eval_rougeL',
                'eval_rougeLsum',
                'eval_bleu',
                'eval_gen_len']

DATE = str(datetime.date.today())

analysis_name = 'NO_domain_2_test' + '-' + DATE
print(analysis_name)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
MODE = 'no_context'

In [ ]:
print(analysis_name)
analysis_modules.bootstrap_inference(analysis_name=analysis_name,
                                    model_name=model_name,
                                    train_data_path=f'../../data/preprocessed/Quantlet/{MODE}/',
                                    train_data_name='full_train_dataset_20230923.json',
                                    val_data_path=f'../../data/preprocessed/Quantlet/20231013/',
                                    val_data_names_list=[f'test_dataset_2_20231013_sample{i}.json' for i in range(5)],
                                    encoder_max_length=encoder_max_length,
                                    decoder_max_length=decoder_max_length,
                                    random_state=RS,
                                    eval_columns_list=EVAL_COLUMNS,
                                    learning_rate=LR,
                                    epochs=EPOCHS,
                                    train_batch=TRAIN_BATCH,
                                    eval_batch=EVAL_BATCH,
                                    warmup_steps=WARMUP_STEPS,
                                    weight_decay=WEIGHT_DECAY,
                                    logging_stes=LOGGING_STEPS,
                                    save_total_lim=SAVE_TOTAL_LIM,
                                    save_strategy=SAVE_STRATEGY,
                                    label_smooting=LABEL_SMOOTHING,
                                    predict_generate=PREDICT_GENERATE,
                                    load_best_model_at_end=load_best_model_at_end,
                                    evaluate_only=True)